# LangChain StructuredOutputParser 예제: 제품 리뷰 분석
`ChatOpenAI`와 `StructuredOutputParser`를 사용해 리뷰 텍스트에서 평점, 장점, 단점 등을 추출합니다.

In [ ]:
# 필수 라이브러리 설치
#!pip install langchain openai

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:10])

gsk_fSXtdh


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

In [4]:
# 출력 구조 정의 (평점, 장점, 단점, 요약)
response_schemas = [
    ResponseSchema(name="rating", description="5점 만점에서 예상 평점"),
    ResponseSchema(name="pros", description="리뷰에서 언급된 장점 3가지를 리스트로 출력"),
    ResponseSchema(name="cons", description="리뷰에서 언급된 단점 3가지를 리스트로 출력"),
    ResponseSchema(name="summary", description="리뷰를 한 문장으로 요약")
]

In [5]:
# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"rating": string  // 5점 만점에서 예상 평점
	"pros": string  // 리뷰에서 언급된 장점 3가지를 리스트로 출력
	"cons": string  // 리뷰에서 언급된 단점 3가지를 리스트로 출력
	"summary": string  // 리뷰를 한 문장으로 요약
}
```


In [6]:
# 프롬프트 템플릿
template = """
다음 제품 리뷰를 분석하세요. 리뷰 내용: {review}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

In [7]:
# 모델 초기화 (temperature=0.5로 설정해 일관성 있는 출력)
#model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

In [8]:
# 테스트 리뷰 데이터
review = """
이 스마트폰은 배터리 수명이 정말 좋아서 하루 종일 사용해도 충전이 필요 없었어요. 
카메라 화질도 선명하고, 특히 야간 모드가 훌륭합니다. 
다만 가격이 조금 비싸고, 무게가 200g이 넘어서 손이 피곤할 수 있어요.
"""

In [9]:
# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"review": review})

In [13]:
# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
print(type(output))
pprint(output)
print(output['summary'])

===== 분석 결과 =====
<class 'dict'>
{'cons': ['가격이 비쌈', '무게 200 g으로 손목에 부담', '무거운 무게로 인한 피로감'],
 'pros': ['하루 종일 사용 가능한 우수한 배터리 수명', '선명한 카메라 화질', '야간 모드 성능이 뛰어남'],
 'rating': '4.5',
 'summary': '배터리와 카메라 성능이 뛰어나지만 가격과 무게가 다소 부담스러운 스마트폰.'}
배터리와 카메라 성능이 뛰어나지만 가격과 무게가 다소 부담스러운 스마트폰.


## 예상 출력 결과
```python
{
    'rating': 4.2,
    'pros': ['배터리 수명 우수', '카메라 화질 선명', '야간 모드 훌륭'],
    'cons': ['가격 비쌈', '무게가 무거움', '손 피로감 가능성'],
    'summary': "배터리와 카메라 성능은 우수하지만, 가격과 무게가 단점인 스마트폰"
}
```